## Mask R-CNN - Inspect Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [ ]:
"""
Based on the work of Waleed Abdulla (Matterport)
Modified by github.com/GustavZ
"""

import os
import sys
import numpy as np
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

# Model  Directory 
MODEL_DIR = os.path.join(ROOT_DIR, "logs")
DEFAULT_WEIGHTS = os.path.join(ROOT_DIR, "mobile_mask_rcnn_coco.h5")

## Configurations

In [ ]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = os.path.join(ROOT_DIR,"data/coco")

In [ ]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()

## Notebook Preferences

In [ ]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
#DEVICE = "/cpu:0"
DEVICE = "/gpu:0"

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"
#TEST_MODE = "training"

In [ ]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Model

In [ ]:
# Local path to trained h5 weights file
MODEL_NAME = 'mask_rcnn_512_cocoperson_0396' # TODO: enter value here
H5_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+".h5") # TODO: enter value here

In [ ]:
import mmrcnn.model as modellib

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR,config=config)

# Set path to model weights
weights_path = DEFAULT_WEIGHTS
#weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

## Prepare and Export Model

In [ ]:
# name of the pb file we want to output
MODEL_NAME = 'mask_rcnn_512_cocoperson_0396' # TODO: enter value here

# Chose whether to quantize the graph
QUANTIZE = False

In [ ]:
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

# Get keras model and save
model_keras= model.keras_model
# All new operations will be in test mode from now on.
K.set_learning_phase(0)

# Create output layer with customized names
num_output = 7
pred_node_names = ["detections", "mrcnn_class", "mrcnn_bbox", "mrcnn_mask", "rois", "rpn_class", "rpn_bbox"]
pred_node_names = ["output_" + name for name in pred_node_names]
pred = [tf.identity(model_keras.outputs[i], name = pred_node_names[i])for i in range(num_output)]

# Get the object detection graph
sess = K.get_session()
if QUANTIZE:
    # Transformations
    transforms = ["quantize_weights", "quantize_nodes"]
    transformed_graph_def = TransformGraph(sess.graph.as_graph_def(), [], pred_node_names, transforms)
    constant_graph = graph_util.convert_variables_to_constants(sess, transformed_graph_def, pred_node_names)
    PB_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+'.pb') 
else:
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
    PB_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+'_quantized'+'.pb') 

In [ ]:
# Write Output pb File
graph_io.write_graph(constant_graph, "/", PB_MODEL_PATH, as_text=False)

# Output Info
print('{} ops in the frozen graph.'.format(len(constant_graph.node)))
print('saved the freezed graph (ready for inference) at: ', PB_MODEL_PATH)

## Inference Model
Now, we can load the model from the pb file and then use it to infere

In [ ]:
# DOES NOT WORK YET

import cv2 
from mmrcnn import utils

utils.set_cuda_visible_devices(config.GPU_COUNT)
with tf.device(DEVICE):
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        output_graph_def = tf.GraphDef()
        with open(PB_MODEL_PATH, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(output_graph_def, name="")

        image = cv2.imread(ROOT_DIR+"/ski.jpk")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(config.IMAGE_MAX_DIM,config.IMAGE_MAX_DIM))
        image = np.expand_dims(image,0)

        image_resized, window, scale,_ = resize_image(image, min_dim=None, max_dim=None, padding=False)
        image_meta = compose_image_meta(image_id=1, image_shape=image_resized.shape, window=window, active_class_ids=[0,1])
        image_meta = np.expand_dims(image_meta,0)
        graph = tf.get_default_graph()
        input_image = graph.get_tensor_by_name("input_image:0")
        input_image_meta = graph.get_tensor_by_name("input_image_meta:0")
        mrcnn_mask = graph.get_tensor_by_name("output_mrcnn_mask:0")

        #[detections, mrcnn_class, mrcnn_bbox, mrcnn_mask,rois, rpn_class, rpn_bbox]
        feed = {input_image:image_resized, input_image_meta:image_meta}
        result = sess.run(mrcnn_mask, feed_dict = feed)
        print(result.shape)